## Importing necessary modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
from tqdm import tqdm # Cool progress bar

import random
import numpy as np
import pandas as pd
import sys
import os # read and manipulate local files

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import cv2
import seaborn as sns

from PIL import Image

import tensorflow.keras as ks
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K # F1-score metric

from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import   Lambda, Conv2D, MaxPool2D, UpSampling2D, BatchNormalization, Flatten
from tensorflow.keras.layers import  GlobalAveragePooling2D, Reshape, Multiply, Attention, add,Resizing,  Input, Dense
from tensorflow.keras.layers import Activation,AveragePooling2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential

# hide wornings
import warnings
warnings.filterwarnings('ignore')

# from keras import ops# For deeplab3

# import keras_cv # basnet

# from imgaug import augmenters as iaa ## for augmentation

## Defining paths

In [ ]:
model_name = 'Attention_UNet_616_v2'

################################################################################################
## SETTING THE PATHS
PATH_TO_CODE ='/content/drive/MyDrive/TRIAL_v1/segmentation_task'
# PATH_TO_CODE =''
sys.path.append(PATH_TO_CODE)

# DATASET_PATH = r'C:\Users\ASUS\Desktop\segmentation & cropping\segmentation_task\segmentation_data43_resized_cropped_split'
DATASET_PATH = r'/content/drive/MyDrive/TRIAL_v1/segmentation_task/segmentation_data43_resized_cropped_split'
# DATASET_PATH = r'segmentation_data43_resized_cropped_split'
DATASET_PATH_TRAIN = os.path.join(DATASET_PATH, 'train')
DATASET_PATH_TEST = os.path.join(DATASET_PATH, 'test')


PATH_TO_SAVE_RESULT = os.path.join(PATH_TO_CODE, 'saved_outputs_segmentation_AUC', model_name)
PATH_BEST_SAVE_WEIGHT = os.path.join(PATH_TO_SAVE_RESULT,'saved_weights')
PATH_TO_SAVE_MODEL = os.path.join(PATH_TO_SAVE_RESULT, 'saved_models')
PATH_SAVE_HISTORY = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_history.csv')
PATH_SAVE_TIME = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_time.csv')
PATH_SAVE_AUGMENT_SAMPLE = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_augmented_sample.pdf')
PATH_SAVE_PIXEL_PERCENTAGE_PLOT = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_pixel_percentage.pdf')
# PATH_TO_SAVE_TUNER = os.path.join(PATH_TO_RESULT, 'saved_tuner_model')
################################################################################################

if not os.path.exists(PATH_TO_SAVE_RESULT):
 # If it does not exist, create it
    os.makedirs(PATH_TO_SAVE_RESULT)
################################################################################################
IMG_HEIGHT = 256
IMG_WIDTH = 256
ORIGINAL_IMAGE_SIZE = (IMG_HEIGHT, IMG_WIDTH)
COLOR_CHANNEL = 3

RESIZE_SHAPE = ORIGINAL_IMAGE_SIZE #(128, 128) #ORIGINAL_IMAGE_SIZE#(128, 128)# # #ORIGINAL_IMAGE_SIZE# (128, 128)#
MODEL_INPUT_SIZE = (RESIZE_SHAPE[0], RESIZE_SHAPE[1], COLOR_CHANNEL)

VALIDATION_SPLIT= 0.2
NUM_CLASSES = 2 # Disease and not disease

BATCH_SIZE = 10
EPOCHS = 33

TOTAL_DATA = 616
################################################################################################

################################################################################################

SAVE_RESULTS = True
SHOW_RESULTS = True

################################################################################################
# Setting the seed
SEED  = 123
RNG = np.random.default_rng(SEED) # Random number generator
tf.random.set_seed(SEED)

################################################################################################
# Checkpoint parameters val_binary_io_u
SCORE_TO_MONITOR = 'val_binary_io_u' # Score that checkpoints monitor during training
SCORE_OBJECTIVE  = 'max'          # 'max' or 'min', specifies whether the objective is to maximize the score or minimize it.
PATIENCE_LR_REDUCE = 3
MIN_LR = 1e-8
REDUCTION_FACTOR = 0.5            # Factor which lr will be reduced with at plateau
# COOLDOWN_EPOCHS  = 2 #cooldown: Integer. Number of epochs to wait before resuming normal operation after the learning rate has been reduced.

INITIAL_LR = 0.001

## Defining performance metrics

Official segmentation metrics by keras: https://ks.io/api/metrics/segmentation_metrics/

In [ ]:
def f1_score(y_true, y_pred): # Dice coefficient
    """
    Calculate the F1 score, the harmonic mean of precision and recall, for binary classification.

    Args:
        y_true (Tensor): True binary labels.
        y_pred (Tensor): Predicted probabilities.

    Returns:
        float32: F1 score as a scalar.
    """
    # True Positives: round product of y_true and y_pred
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    # Actual Positives: round y_true
    P = K.sum(K.round(K.clip(y_true, 0, 1)))
    # Recall: TP / Actual Positives
    recall = TP / (P + K.epsilon())

    # Predicted Positives: round y_pred
    Pred_P = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # Precision: TP / Predicted Positives
    precision = TP / (Pred_P + K.epsilon())

    # F1 Score: harmonic mean of precision and recall
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# source: https://www.tensorflow.org/api_docs/python/tf/keras/metrics/BinaryIoU
iou_score_binary = tf.keras.metrics.BinaryIoU(target_class_ids=[0, 1], threshold=0.5)

def accuracy_score(y_true, y_pred):
    """
    Calculate accuracy score between two binary masks.
    """
    correct = np.sum(y_true == y_pred)  # Count correct predictions
    total = y_true.size  # Total number of pixels
    return correct / total  # Accuracy calculation

def precision_score(groundtruth_mask, pred_mask):
    """
    Calculate precision score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_pred = np.sum(pred_mask)  # Sum of predicted positives
    return np.mean(intersect / total_pixel_pred)  # Precision calculation

def recall_score(groundtruth_mask, pred_mask):
    """
    Calculate recall score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_truth = np.sum(groundtruth_mask)  # Sum of actual positives
    return np.mean(intersect / total_pixel_truth)  # Recall calculation

## Reading the data

In [ ]:
# Function to read images from a directory using Pillow
def read_images_from_directory(directory_path):
    images = []
    for filename in sorted(os.listdir(directory_path)):
        # filename= filename.lower()

        if filename.endswith(('.png', '.jpg', '.JPG' )):  # Check for image file extensions
            img_path = os.path.join(directory_path, filename)
            img = Image.open(img_path)
            img_array = np.array(img)  # Convert the image to a numpy array if needed
            if img_array is not None:
                images.append(img_array)
    return images

# Paths to masks and original images within the dataset
masks_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_masks_binarised')
originals_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_original')

# Read images into variables
x_train = read_images_from_directory(originals_path_train)  # Original images
y_train = read_images_from_directory(masks_path_train)  # Masks

# Paths to masks and original images within the test dataset
masks_path_test = os.path.join(DATASET_PATH_TEST, 'data43_masks_binarised')
originals_path_test = os.path.join(DATASET_PATH_TEST, 'data43_original')

# Read images into variables
x_test = read_images_from_directory(originals_path_test)  # Original images
y_test = read_images_from_directory(masks_path_test)  # Masks

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 616 original images into x_train
Loaded 616 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Preprocessing DO NOT RESIZE THE IMAGE-- IT GIVES FINE LINES IN THE OUTPUT

In [ ]:
# # # Resizing
# x_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_train]
# y_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_train]

# x_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_test]
# y_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_test]

# Convert the list to a NumPy array
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Expand mask dimensions to (batch, 256, 256, 1)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

# Normalizing input between [0,1]
x_train = x_train.astype("float32")/ np.max(x_train)
x_test  = x_test.astype("float32")/np.max(x_test)

y_train = y_train.astype("float32")/ np.max(y_train)
y_test  = y_test.astype("float32")/np.max(y_test)

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 616 original images into x_train
Loaded 616 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Essential functions

In [ ]:
def fit_and_save_best_model(model_name, model, epochs=EPOCHS):
    ##########################################################################################################
    saved_best_model_name = 'best_'+model_name+'.h5'

    model_saving_path = os.path.join(PATH_TO_SAVE_MODEL, saved_best_model_name)
    # Set up a model checkpoint to save the best model during training
    best_model_callback= ModelCheckpoint(model_saving_path,
                                          monitor=SCORE_TO_MONITOR,
                                          save_best_only=True,
                                          mode=SCORE_OBJECTIVE,
                                          verbose=1)

    reduce_LR = ReduceLROnPlateau(
        factor=REDUCTION_FACTOR,      # Factor by which the learning rate will be reduced. new_lr = lr * factor
        patience=PATIENCE_LR_REDUCE, # original was 5      # Number of epochs with no improvement after which learning rate will be reduced.
        verbose=1,       # int. 0: quiet, 1: update messages.
        min_lr=MIN_LR   # Lower bound on the learning rate.
    )


    my_callbacks = [best_model_callback , reduce_LR]
    ##########################################################################################################

    start_time = time.time()

    # Fitting the model
    train_history = model.fit(
        x_train,
        y_train,
        epochs= epochs,
        batch_size=BATCH_SIZE,
        validation_split= VALIDATION_SPLIT,
        callbacks=my_callbacks,
    )
    total_time = time.time() - start_time

    return saved_best_model_name, train_history, total_time, model

## Models
### Models worked
- Model01: Basic basic_unet
- Model02: unet_vgg16 (my own custom)
- Model03: TransUNet
- Model04: DeepLab-v3Plus
- Model05: ResUNet
- Model06: UNet_with_attention
- Model07: Multi-resUnet
- Model08: Inception_resnetV2 (my own custom)



### UNet_with_attention--works--Ran with batch size10 (memory issues)
---Works properly after 10 epochs--within 5 epoch--sometimes does not show anything

In [ ]:
def conv_block(x, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(x, num_filters):
    x = conv_block(x, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def attention_gate(g, s, num_filters):
    Wg = Conv2D(num_filters, 1, padding="same")(g)
    Wg = BatchNormalization()(Wg)

    Ws = Conv2D(num_filters, 1, padding="same")(s)
    Ws = BatchNormalization()(Ws)

    out = Activation("relu")(Wg + Ws)
    out = Conv2D(num_filters, 1, padding="same")(out)
    out = Activation("sigmoid")(out)

    return out * s

def decoder_block(x, s, num_filters):
    x = UpSampling2D(interpolation="bilinear")(x)
    s = attention_gate(x, s, num_filters)
    x = Concatenate()([x, s])
    x = conv_block(x, num_filters)
    return x

def UNet_with_attention():
    """ Inputs """

    ### Source: https://github.com/nikhilroxtomar/Semantic-Segmentation-Architecture/blob/main/TensorFlow/attention-unet.py
    inputs = Input(MODEL_INPUT_SIZE)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)

    b1 = conv_block(p3, 512)

    """ Decoder """
    d1 = decoder_block(b1, s3, 256)
    d2 = decoder_block(d1, s2, 128)
    d3 = decoder_block(d2, s1, 64)

    """ Outputs """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d3)

    """ Model """
    model = Model(inputs, outputs, name="Attention-UNET")
    optimizer = Adam(learning_rate=INITIAL_LR)
    model.compile(loss='binary_crossentropy',
                  metrics=['accuracy', f1_score, iou_score_binary],
                  optimizer=optimizer)

    return model

## Train and Results

In [ ]:
# model= get_basic_unet() ## Model1
# model= get_unet_vgg16() ## Model2
# model= get_TransUNet() ## Model3
# model= get_DeepLabV3Plus() ## Model4
# model = get_ResUNet_v3() ## Model5
model = UNet_with_attention() ## Model6
# model = get_multiresunet() ## Model7
# model = get_unet_InceptionResNetV2() ## Model8

model.summary()

Model: "Attention-UNET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]  

 Total params: 8,143,169 (31.06 MB)

 Trainable params: 8,135,745 (31.04 MB)

 Non-trainable params: 7,424 (29.00 KB)

In [ ]:
# training the model and saving the best model as a check point
best_model_name, train_history, total_time, best_model = fit_and_save_best_model(model_name, model)

Epoch 1/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8316 - binary_io_u: 0.4862 - f1_score: 0.3296 - loss: 0.4027   
Epoch 1: val_binary_io_u improved from -inf to 0.06482, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Attention_UNet_616_v2/saved_models/best_Attention_UNet_616_v2.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 155s 2s/step - accuracy: 0.8336 - binary_io_u: 0.4881 - f1_score: 0.3326 - loss: 0.3996 - val_accuracy: 0.1223 - val_binary_io_u: 0.0648 - val_f1_score: 0.0897 - val_loss: 1.9570 - learning_rate: 0.0010
Epoch 2/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 687ms/step - accuracy: 0.9742 - binary_io_u: 0.7479 - f1_score: 0.6757 - loss: 0.1144
Epoch 2: val_binary_io_u improved from 0.06482 to 0.42280, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Attention_UNet_616_v2/saved_models/best_Attention_UNet_616_v2.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 39s 782ms/step - accuracy: 0.9742 - binary_io_u: 0.7480 - f1_score: 0.6759 - loss: 0.1143 - val_accuracy: 0.8368 - val_binary_io_u: 0.4228 - val_f1_score: 0.0187 - val_loss: 0.4490 - learning_rate: 0.0010
Epoch 3/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 656ms/step - accuracy: 0.9777 - binary_io_u: 0.7737 - f1_score: 0.7187 - loss: 0.0882
Epoch 3: val_binary_io_u improved from 0.42280 to 0.48821, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Attention_UNet_616_v2/saved_models/best_Attention_UNet_616_v2.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 37s 749ms/step - accuracy: 0.9777 - binary_io_u: 0.7737 - f1_score: 0.7187 - loss: 0.0881 - val_accuracy: 0.9555 - val_binary_io_u: 0.4882 - val_f1_score: 0.0311 - val_loss: 0.1645 - learning_rate: 0.0010
Epoch 4/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 665ms/step - accuracy: 0.9783 - binary_io_u: 0.7798 - f1_score: 0.7289 - loss: 0.0767
Epoch 4: val_binary_io_u improved from 0.48821 to 0.55660, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Attention_UNet_616_v2/saved_models/best_Attention_UNet_616_v2.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 47s 942ms/step - accuracy: 0.9783 - binary_io_u: 0.7797 - f1_score: 0.7287 - loss: 0.0767 - val_accuracy: 0.9614 - val_binary_io_u: 0.5566 - val_f1_score: 0.2055 - val_loss: 0.1325 - learning_rate: 0.0010
Epoch 5/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 674ms/step - accuracy: 0.9785 - binary_io_u: 0.7823 - f1_score: 0.7305 - loss: 0.0719
Epoch 5: val_binary_io_u improved from 0.55660 to 0.57431, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Attention_UNet_616_v2/saved_models/best_Attention_UNet_616_v2.h5


...

50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 760ms/step - accuracy: 0.9829 - binary_io_u: 0.8211 - f1_score: 0.7858 - loss: 0.0523 - val_accuracy: 0.9820 - val_binary_io_u: 0.8055 - val_f1_score: 0.7132 - val_loss: 0.0604 - learning_rate: 0.0010
Epoch 14/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 664ms/step - accuracy: 0.9833 - binary_io_u: 0.8245 - f1_score: 0.7898 - loss: 0.0513
Epoch 14: val_binary_io_u did not improve from 0.80553
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 724ms/step - accuracy: 0.9833 - binary_io_u: 0.8246 - f1_score: 0.7898 - loss: 0.0513 - val_accuracy: 0.9795 - val_binary_io_u: 0.7907 - val_f1_score: 0.7060 - val_loss: 0.0647 - learning_rate: 0.0010
Epoch 15/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 662ms/step - accuracy: 0.9837 - binary_io_u: 0.8282 - f1_score: 0.7931 - loss: 0.0506
Epoch 15: val_binary_io_u did not improve from 0.80553
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 722ms/step - accuracy: 0.9837 - binary_io_u: 0.8283 - f1_score: 0.7932 - loss: 0.0507 - val_accuracy: 0.9793 - val_binary_io_u: 0.7818 - va

50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 758ms/step - accuracy: 0.9877 - binary_io_u: 0.8656 - f1_score: 0.8428 - loss: 0.0377 - val_accuracy: 0.9817 - val_binary_io_u: 0.8126 - val_f1_score: 0.7369 - val_loss: 0.0575 - learning_rate: 1.2500e-04
Epoch 25/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 667ms/step - accuracy: 0.9879 - binary_io_u: 0.8669 - f1_score: 0.8447 - loss: 0.0369
Epoch 25: val_binary_io_u improved from 0.81264 to 0.81378, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Attention_UNet_616_v2/saved_models/best_Attention_UNet_616_v2.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 761ms/step - accuracy: 0.9879 - binary_io_u: 0.8670 - f1_score: 0.8448 - loss: 0.0369 - val_accuracy: 0.9819 - val_binary_io_u: 0.8138 - val_f1_score: 0.7368 - val_loss: 0.0569 - learning_rate: 1.2500e-04
Epoch 26/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 661ms/step - accuracy: 0.9881 - binary_io_u: 0.8689 - f1_score: 0.8473 - loss: 0.0362
Epoch 26: val_binary_io_u did not improve from 0.81378
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 721ms/step - accuracy: 0.9881 - binary_io_u: 0.8689 - f1_score: 0.8473 - loss: 0.0362 - val_accuracy: 0.9818 - val_binary_io_u: 0.8127 - val_f1_score: 0.7359 - val_loss: 0.0573 - learning_rate: 1.2500e-04
Epoch 27/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 663ms/step - accuracy: 0.9883 - binary_io_u: 0.8708 - f1_score: 0.8497 - loss: 0.0354
Epoch 27: val_binary_io_u did not improve from 0.81378
50/50 ━━━━━━━━━━━━━━━━━━━━ 36s 723ms/step - accuracy: 0.9883 - binary_io_u: 0.8709 - f1_score: 0.8497 - loss: 0.0354 - val_accuracy: 0.9818 - val_binary_io_u: 0.8

50/50 ━━━━━━━━━━━━━━━━━━━━ 40s 795ms/step - accuracy: 0.9889 - binary_io_u: 0.8778 - f1_score: 0.8584 - loss: 0.0327 - val_accuracy: 0.9819 - val_binary_io_u: 0.8167 - val_f1_score: 0.7444 - val_loss: 0.0581 - learning_rate: 3.1250e-05
Epoch 33/33
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 668ms/step - accuracy: 0.9890 - binary_io_u: 0.8793 - f1_score: 0.8605 - loss: 0.0323
Epoch 33: val_binary_io_u improved from 0.81670 to 0.81699, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Attention_UNet_616_v2/saved_models/best_Attention_UNet_616_v2.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 762ms/step - accuracy: 0.9890 - binary_io_u: 0.8794 - f1_score: 0.8605 - loss: 0.0323 - val_accuracy: 0.9819 - val_binary_io_u: 0.8170 - val_f1_score: 0.7451 - val_loss: 0.0580 - learning_rate: 3.1250e-05


## Loading the best model

In [ ]:
# # ##Now, load the best model
# # best_model = ks.models.load_model(os.path.join(PATH_TO_SAVE_MODEL, best_model_name),
# #                                   custom_objects={'f1_score': f1_score,
# #                                                   'binary_io_u':iou_score_binary,
# #                                                   "accuracy_score":accuracy_score,
# #                                                   "precision_score":precision_score,
# #                                                   "recall_score":recall_score})

# best_model = ks.models.load_model(
#     os.path.join(PATH_TO_SAVE_MODEL, best_model_name),
#     custom_objects={
#         'binary_io_u': iou_score_binary
#     },
#     compile=False  # avoids reinitializing missing optimizer/metrics
# )

# # # best_model =model

# # ##Now, load the best model
# # best_model = ks.models.load_model(os.path.join(PATH_TO_SAVE_MODEL, best_model_name),
# #                                   custom_objects={'f1_score': f1_score,
# #                                                   'binary_io_u': tf.keras.metrics.BinaryIoU}) # Pass the class instead of the instance

# # custom_objects = {
# #     'f1_score': f1_score,
# #     'binary_io_u': iou_score_binary,
# #     'attention_gate': attention_gate,
# #     'conv_block': conv_block,
# #     'encoder_block': encoder_block,
# #     'decoder_block': decoder_block,
# # }

# # best_model = ks.models.load_model(os.path.join(PATH_TO_SAVE_MODEL, best_model_name))

# # best_model = ks.models.load_model( "/content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Attention_UNet_616_v2/saved_models/best_Attention_UNet_616_v2.h5",
# #                                   custom_objects={'f1_score': f1_score,
# #                                                   'binary_io_u':iou_score_binary})


## Visualize model predictions

In [ ]:
# Get predictions from the model
predictions = best_model.predict(x_test, verbose=1)

USER_DETERMINED_THRESHOLD = 0.5
thresholded_predictions  = (predictions  >= USER_DETERMINED_THRESHOLD)
# Removing the color channel
thresholded_predictions_without_color_channel = np.squeeze(thresholded_predictions, axis=-1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 31s 31s/step


# AUC

In [ ]:
from sklearn.metrics import roc_auc_score

# Flatten masks for AUC computation
y_true_flat = y_test.flatten()
y_score_flat = predictions.flatten()  # Predicted probabilities

In [ ]:
# Bootstrapping AUC
n_iterations = 100
rng = np.random.default_rng(SEED)
bootstrap_aucs = []

for _ in range(n_iterations):
    indices = rng.integers(0, len(y_true_flat), len(y_true_flat))
    y_true_sample = y_true_flat[indices]
    y_score_sample = y_score_flat[indices]

    # Ensure both classes are present in the sample
    if len(np.unique(y_true_sample)) < 2:
        continue

    auc = roc_auc_score(y_true_sample, y_score_sample)
    bootstrap_aucs.append(auc)

In [ ]:
# Calculate statistics
mean_auc = np.mean(bootstrap_aucs)
std_auc = np.std(bootstrap_aucs)
lower_ci = np.percentile(bootstrap_aucs, 2.5)
upper_ci = np.percentile(bootstrap_aucs, 97.5)

# Print AUC statistics
print(f"Bootstrapped AUC: Mean={mean_auc:.4f}, Std={std_auc:.4f}, 95% CI=({lower_ci:.4f}, {upper_ci:.4f})")

# Optional saving
if SAVE_RESULTS:
    auc_df = pd.DataFrame({'AUC Scores': bootstrap_aucs})
    auc_df.to_csv(os.path.join(PATH_TO_SAVE_RESULT, f"{model_name}_bootstrapped_auc.csv"), index=False)

Bootstrapped AUC: Mean=0.9768, Std=0.0004, 95% CI=(0.9760, 0.9775)
